## Final Project Submission

* Student name:                        KEVIN GIROUX
* Student pace:                        FULL TIME
* Scheduled project review date/time:  (TBD)
* Instructor name:                     JAMES IRVING
* Blog post URL:                       (TBD)


# Introduction

# Review of provided data; gap analysis

In [1]:
import pandas as pd
import json

In [2]:
# general movie metadata
imdb_df = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
rt_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t')
title_aka_df = pd.read_csv('zippedData/imdb.title.akas.csv.gz')

# ratings and review data
ratings_df = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
tmdb_ratings_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t', encoding='latin1')

# people
ppl_df = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
crew_df = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
principals_df = pd.read_csv('zippedData/imdb.title.principals.csv.gz')

# financials
gross_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
budget_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

## Qualitative description of data sets:


### a. Movie background info and metadata
- imdb_df:  (IMDB) movie titles (w/ IMDB title id) by release year, runtime, and genre
- rt_df:  (Rotten Tomatoes) synopsis, rating, genre, director, writer, runtime, etc.
- title_aka_df:  (IMDB) movie titles (w/ IMDB title id) in different languages

### b. Reviews and ratings
- ratings_df:  (IMDB) average rating and vote count, by IMDB title id
- tmdb_ratings_df:  (TMDB) popularity score, with vote counts and average vote score, by title 
- rt_reviews_df:  (Rotten Tomatoes) includes review text, numerical rating, and review publisher

### c. People involved
- ppl_df:  (IMDB) info on humans involved, such as birth year, primary profession, and movies they are known for (by title id)
- crew_df:  (IMDB) directors and writers, by movie id (matches title id with name id)
- principals_df:  (IMDB) (title id, name id) categorized list of people by role/job in film production (includes list of characters for actors/actresses)

### d. Financial
- gross_df:  (BOM) domestic and foreign gross rev by movie title, studio, and year
- budget_df: (TN) production budget vs. domestic and worldwide gross rev, by movie title (no movie id!)


##### Below, I will examine each of the above sub-sets of data in more detail and draw conclusions on which data sets/features to use and how

## a. Movie background info and metadata
- imdb_df: 'genres' column contains list of multiple genres separated by comma (no spaces)
- rt_df: no movie titles are included, just an RT movie id;  'rating', 'director', 'writer', 'theater_date', 'studio', 'box_office' are all unique features to RT data
- title_aka_df: not sure I will need this data


##### CONCLUSIONS: 
- Use imdb_df as my base data set
- look for ways to pull in 'rating', 'studio', 'theater_date' and 'box_office' from rt_df or the internet

In [3]:
imdb_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
rt_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [5]:
title_aka_df.head()

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


## b. Reviews and ratings
- ratings_df: imdb title id, average score and number of votes...needs enrichment
- tmdb_ratings_df: has a lot of cool features like a popularity score, release date, vote_average, vote_count, original_language -- can by joined by 'original_title' or 'title'
- rt_reviews_df: again, no movie titles included, includes a top critic designation which is cool, and a fresh/not fresh binary categorization column, also 'publisher' columns

##### CONCLUSIONS:
- use ratings_df as my starting point for reviews and ratings data, because of easy join-ability with movie metadata via IMDB title id
- find way to enrich this review data via API/web-scraping or by joining some features from the TMDB dataset, which has a lot of features that would be interesting to play with ('original language', 'popularity', 'vote_average', 'vote count')
- RT data has a couple interesting features, but no movie title column will make this data very hard to join with my other rating/review data

In [6]:
ratings_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [7]:
tmdb_ratings_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [8]:
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


## c. People involved
- ppl_df: full list of people in film industry, with unique imdb name ids, known_for_titles by imdb title id
- crew_df: imdb name ids for directors and writers, associated with imdb film ids
- principals_df: imdb film ids, with imdb name ids of principal resources associated, by job function

##### CONCLUSIONS:
- principals_df could be useful for developing recommendations for specific writers, directors, actors/actresses, etc. to use
- may need to use ppl_df and crew_df to perform any necessary joins between people data and our other data sub-sets (movie metadata, ratings/reviews, and financials)
- all of these are from IMDB and use IMDB's title ID and name ID, which should make for easy joining

In [9]:
ppl_df.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [10]:
crew_df.head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [11]:
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


## d. Financials
- gross_df: includes a feature for studio abbreviations and a feature for the year the film was released
- budget_df: includes a budget feature, along with revenue data, as well as a full 'release_date' feature

##### CONCLUSIONS:
- this will be the most challenging sub-set of data to join with the rest, as neither of the provided financial datasets is from IMDB
- leaning towards using budget_df exclusively for my financial data at this point, due to inclusion of 'budget' feature and full 'release_date' feature rather than just the release year
- hopefully there will be a more direct way to scrape and integrate these numbers from the internet

In [12]:
gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [13]:
budget_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Gap analysis

- Use imdb_df as my base data set for movie metadata
    - look for ways to pull in 'rating', 'studio', 'theater_date' and 'box_office' from rt_df or from the internet


- Use ratings_df as my starting point for reviews and ratings data, because of easy join-ability with movie metadata via IMDB title id
    - join in 'original language', 'popularity', 'vote_average', 'vote count' from tmdb_ratings_df or internet
    - Unlikely to use RT data due to lack of movie title feature


- Use principals_df for developing recommendations on specific writers, directors, actors/actresses, etc. to use (join with metadata via IMDB ID)
    - may need to use ppl_df and crew_df to complete the joins with final combined dataset


- sources of financial provided are NOT from IMDB, so cannot join financials via IMDB ID as above
    - budget_df includes 'budget' along with gross totals, so this is the better of the two provided sets of data
    - finding ways to scrape financials from web will be a top priority 

# Cleaning and tidying

In [14]:
# I know these data sets are messy, so before attempting to clean I jumped online and found a Python toolkit called
# IMDbPY that was specifically designed to query IMDb using python.  I will import below and see what gaps I can fill
# in imdb_df using the API before attempting any joins.

## Exploring IMDbPY Python toolkit

In [3]:
import imdb
ia = imdb.IMDb()

In [16]:
# List of the sets of features we can import using IMDbPY
ia.get_movie_infoset()

['airing',
 'akas',
 'alternate versions',
 'awards',
 'connections',
 'crazy credits',
 'critic reviews',
 'episodes',
 'external reviews',
 'external sites',
 'faqs',
 'full credits',
 'goofs',
 'keywords',
 'list',
 'locations',
 'main',
 'misc sites',
 'news',
 'official sites',
 'parents guide',
 'photo sites',
 'plot',
 'quotes',
 'recommendations',
 'release dates',
 'release info',
 'reviews',
 'sound clips',
 'soundtrack',
 'synopsis',
 'taglines',
 'technical',
 'trivia',
 'tv schedule',
 'video clips',
 'vote details']

In [17]:
# List of features within the feature set 'main'
# NOTE Moonlight (2016) aka 'tt4975722' used as placeholder for all following queries of IMDbPY
ia.get_movie('4975722', info='main').keys()

['original title',
 'cast',
 'genres',
 'runtimes',
 'countries',
 'country codes',
 'language codes',
 'color info',
 'aspect ratio',
 'sound mix',
 'box office',
 'certificates',
 'original air date',
 'rating',
 'votes',
 'cover url',
 'imdbID',
 'plot outline',
 'languages',
 'title',
 'year',
 'kind',
 'directors',
 'writers',
 'producers',
 'composers',
 'cinematographers',
 'editors',
 'editorial department',
 'casting directors',
 'production designers',
 'art directors',
 'set decorators',
 'costume designers',
 'make up department',
 'production managers',
 'assistant directors',
 'art department',
 'sound department',
 'visual effects',
 'stunts',
 'camera department',
 'casting department',
 'costume departmen',
 'location management',
 'music department',
 'script department',
 'miscellaneous',
 'thanks',
 'akas',
 'writer',
 'director',
 'production companies',
 'distributors',
 'special effects',
 'other companies',
 'canonical title',
 'long imdb title',
 'long imdb can

### Programmatic exploration of toolkit
#### a. Movie metadata
- movie rating:  IMDbPY 'main' 'certificates' (this lists theater rating for all countries, need to isolate US)
- studio:  IMDbPY 'main' 'production companies'
- theater date:  IMDbPY 'main' 'original air date' AND 'year'
- PLUS original language:  this could be pulled in from TMDB_ratings_df as well

#### b. Ratings and reviews
- IMDbPY 'vote details' 'demographics' for demographic info about top voters of movies)
- PLUS still need to join in TMDB_ratings_df for 'popularity', 'vote_average', 'vote_count'

#### c. People
- FIRST I want to explore principals_df, but if that proves to be too complicated for joining....
- if not, I can at least pull 'writer' and 'director' data from IMDbPY

#### d. Financials
- IMDbPY 'main' 'box office' (includes budget, opening weekend gross, domestic gross, global gross)

In [18]:
ia.get_movie('4975722', info='main')['certificates']

['Argentina:16',
 'Australia:M',
 'Austria:16',
 'Brazil:16',
 'Canada:14A::(Alberta/British Columbia/Manitoba)',
 'Canada:13+::(Quebec)',
 'Chile:14',
 'Colombia:12',
 'Denmark:11',
 'Finland:K-12',
 'France:Tous publics',
 'Germany:12',
 'Hong Kong:IIB',
 'Hungary:16',
 'India:A',
 'Indonesia:17+',
 'Ireland:15A',
 'Italy:T',
 'Japan:R15+',
 'Lithuania:N-13',
 'Malaysia:(Banned)',
 'Malta:15',
 'Mexico:B15',
 'Netherlands:12',
 'New Zealand:M',
 'Norway:12',
 'Philippines:R-16',
 'Portugal:M/16',
 'Singapore:M18',
 'South Africa:13',
 'South Korea:15',
 'Spain:16',
 'Sweden:11',
 'Switzerland:14',
 'Taiwan:R-15',
 'Turkey:15+',
 'United Kingdom:15',
 'United States:R',
 'Ukraine:16']

In [19]:
ia.get_movie('4975722', info='main')['production companies']

[<Company id:0390816[http] name:_A24_>,
 <Company id:0641956[http] name:_PASTEL_>,
 <Company id:0136967[http] name:_Plan B Entertainment_>]

In [20]:
ia.get_movie('4975722', info='main')['box office']

{'Budget': '$1,500,000 (estimated)',
 'Opening Weekend United States': '$1,488,740, 18 Nov 2016',
 'Cumulative Worldwide Gross': '$55,561,162, 20 Mar 2017'}

In [21]:
ia.get_movie('4975722', info='vote details')['demographics']

{'imdb users': {'votes': 261568, 'rating': 7.4},
 'aged under 18': {'votes': 295, 'rating': 7.9},
 'aged 18 29': {'votes': 68729, 'rating': 7.6},
 'aged 30 44': {'votes': 83967, 'rating': 7.2},
 'aged 45 plus': {'votes': 20008, 'rating': 7.2},
 'males': {'votes': 146977, 'rating': 7.3},
 'males aged under 18': {'votes': 203, 'rating': 8.0},
 'males aged 18 29': {'votes': 50778, 'rating': 7.6},
 'males aged 30 44': {'votes': 67196, 'rating': 7.2},
 'males aged 45 plus': {'votes': 15858, 'rating': 7.1},
 'females': {'votes': 38969, 'rating': 7.6},
 'females aged under 18': {'votes': 66, 'rating': 7.8},
 'females aged 18 29': {'votes': 16079, 'rating': 7.7},
 'females aged 30 44': {'votes': 15253, 'rating': 7.5},
 'females aged 45 plus': {'votes': 3702, 'rating': 7.5},
 'top 1000 voters': {'votes': 565, 'rating': 6.7},
 'us users': {'votes': 32764, 'rating': 7.8},
 'non us users': {'votes': 105093, 'rating': 7.3}}

In [22]:
ia.get_movie('4975722', info='main')['writer']

[<Person id:1503575[http] name:_Barry Jenkins_>,
 <Person id:4144120[http] name:_Tarell Alvin McCraney_>]

In [23]:
ia.get_movie('4975722', info='main')['director']

[<Person id:1503575[http] name:_Barry Jenkins_>]

### CONCLUSIONS

- I will join imdb_df, ratings_df, and principals_df to start forming a fully combined dataframe for analysis
- almost all interesting missing metadata can be pulled in from IMDbPY, including financials
- TMDB_ratings_df is the only non-IMDB dataset I need to focus on joining in

## Joining IMDB datasets

### Metadata + ratings = full_df

In [24]:
# First I will left-join imdb_df and ratings_df on 'tconst' to form full_df, my main data set for analysis

imdb_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [25]:
ratings_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [4]:
full_df = pd.merge(imdb_df, ratings_df, on='tconst', how='left')

### full_df += people data

In [27]:
# Now I will examine principals_df to see how I can join this info into full_df

principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [28]:
# I will need to unmelt principals_df to get one single row for each film with separate columns 
# for each of the below categories containing the nconst ID where applicable

principals_df.category.value_counts()

actor                  256718
director               146393
actress                146208
producer               113724
cinematographer         80091
composer                77063
writer                  74357
self                    65424
editor                  55512
production_designer      9373
archive_footage          3307
archive_sound              16
Name: category, dtype: int64

In [5]:
# I will drop the unwanted columns 'ordering', 'job', and 'characters'
principals_df2 = principals_df.copy()
principals_df2.drop(['ordering', 'job', 'characters'], axis = 1, inplace=True)
principals_df2.head()

,tconst,nconst,category
0,tt0111414,nm0246005,actor
1,tt0111414,nm0398271,director
2,tt0111414,nm3739909,producer
3,tt0323808,nm0059247,editor
4,tt0323808,nm3579312,actress


In [6]:
principals_df2.pivot(index='tconst', columns='category', values='nconst')

ValueError: Index contains duplicate entries, cannot reshape

In [7]:
# Pivoting on the above dataframe produces a duplicate entries error, which I suspect means there are duplicate 
# tconst - category pairings for certain tconst ID's (i.e. multiple actors listed, each as its own row, for the 
# same movie)

# I will first drop all categories that I'm not interested in joining with full_df
# Then I will investigate the duplicates issue

principals_df2 = principals_df2[(principals_df2.category == 'actor') | 
                                (principals_df2.category == 'director') | 
                                (principals_df2.category == 'producer') | 
                                (principals_df2.category == 'editor') | 
                                (principals_df2.category == 'actress') | 
                                (principals_df2.category == 'writer') | 
                                (principals_df2.category == 'composer')]

In [8]:
# Unfortunately, I was correct about the duplicates issue described above, based on below example
principals_df3 = principals_df2.drop('nconst', axis=1)
principals_df3[principals_df3.tconst == 'tt0323808']

,tconst,category
3,tt0323808,editor
4,tt0323808,actress
5,tt0323808,actor
6,tt0323808,actor
7,tt0323808,actress
8,tt0323808,director
9,tt0323808,producer
10,tt0323808,producer
11,tt0323808,composer


In [9]:
# To resolve the duplicate row problem, I will group principals_df2 by tconst and category, and aggregate 
# the matching nconst values into a single cell (when there are multiple), in order to execute a pivot

principals_df4 = principals_df2.groupby(['tconst', 'category'], as_index=False).agg({'nconst':lambda x: ', '.join(tuple(x.tolist()))})


In [10]:
# Nice - this worked, we have combined duplicate category items into a single cell for each tconst ID
principals_df4.head()

,tconst,category,nconst
0,tt0063540,actor,"nm0474801, nm0756379, nm0474876"
1,tt0063540,actress,nm0904537
2,tt0063540,composer,nm0006210
3,tt0063540,director,nm0712540
4,tt0063540,writer,"nm0023551, nm1194313, nm0347899, nm1391276"


In [11]:
# Pivoting on the above, I can now join each of the below "people" columns with full_df by using tconst
principals_piv = principals_df4.pivot(index='tconst', columns='category', values='nconst')
principals_piv = principals_piv.reset_index()
principals_piv.head()

category,tconst,actor,actress,composer,director,editor,producer,writer
0,tt0063540,"nm0474801, nm0756379, nm0474876",nm0904537,nm0006210,nm0712540,NaN,NaN,"nm0023551, nm1194313, nm0347899, nm1391276"
1,tt0066787,"nm0451809, nm0794511","nm0045119, nm0754829",nm2600399,nm0002411,NaN,NaN,NaN
2,tt0069049,"nm0001379, nm0000953","nm0462648, nm0001782",nm0006166,nm0000080,nm0613657,"nm0550881, nm1475059",NaN
3,tt0069204,"nm0315917, nm0037026, nm2147526, nm0244884","nm1027929, nm1875977, nm1877741",nm0006005,nm0611531,NaN,NaN,nm0347899
4,tt0100275,"nm0016013, nm0721280","nm0728971, nm1415193",nm0005948,"nm0749914, nm0765384",NaN,"nm0462571, nm1856792",nm1360635


In [12]:
full_df = full_df.merge(principals_piv, on='tconst', how='left')

In [13]:
full_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,actress,composer,director,editor,producer,writer
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,"nm0474801, nm0756379, nm0474876",nm0904537,nm0006210,nm0712540,NaN,NaN,"nm0023551, nm1194313, nm0347899, nm1391276"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,"nm0451809, nm0794511","nm0045119, nm0754829",nm2600399,nm0002411,NaN,NaN,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,"nm0001379, nm0000953","nm0462648, nm0001782",nm0006166,nm0000080,nm0613657,"nm0550881, nm1475059",NaN
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,"nm0315917, nm0037026, nm2147526, nm0244884","nm1027929, nm1875977, nm1877741",nm0006005,nm0611531,NaN,NaN,nm0347899
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0016013, nm0721280","nm0728971, nm1415193",nm0005948,"nm0749914, nm0765384",NaN,"nm0462571, nm1856792",nm1360635


## Initial IMDB data clean

In [14]:
# Before I query IMDbPY, I want to thin out the data included in full_df, to minimize query execution time
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146144 entries, 0 to 146143
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   averagerating    73856 non-null   float64
 7   numvotes         73856 non-null   float64
 8   actor            95162 non-null   object 
 9   actress          77333 non-null   object 
 10  composer         63521 non-null   object 
 11  director         128436 non-null  object 
 12  editor           48950 non-null   object 
 13  producer         70319 non-null   object 
 14  writer           49297 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 17.8+ MB


In [15]:
# To start, I will drop any rows that don't have a value for "averagerating" or "numvotes"
full_df.dropna(subset=['numvotes'], axis=0, inplace=True)

In [16]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 146134
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  float64
 8   actor            58001 non-null  object 
 9   actress          49952 non-null  object 
 10  composer         41547 non-null  object 
 11  director         67757 non-null  object 
 12  editor           28664 non-null  object 
 13  producer         46674 non-null  object 
 14  writer           32894 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 9.0+ MB


In [17]:
# Now I will drop any rows without runtime_minutes 
full_df.dropna(subset=['runtime_minutes'], axis=0, inplace=True)

In [18]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66236 entries, 0 to 146134
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           66236 non-null  object 
 1   primary_title    66236 non-null  object 
 2   original_title   66236 non-null  object 
 3   start_year       66236 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           65720 non-null  object 
 6   averagerating    66236 non-null  float64
 7   numvotes         66236 non-null  float64
 8   actor            51825 non-null  object 
 9   actress          44617 non-null  object 
 10  composer         38200 non-null  object 
 11  director         60983 non-null  object 
 12  editor           25983 non-null  object 
 13  producer         42762 non-null  object 
 14  writer           29812 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 8.1+ MB


In [19]:
full_df.numvotes.value_counts()

6.0         2281
5.0         2134
7.0         1962
8.0         1739
9.0         1551
            ... 
6380.0         1
9167.0         1
3843.0         1
179453.0       1
4176.0         1
Name: numvotes, Length: 7349, dtype: int64

In [20]:
# To further shave down the data set, I will remove all titles that received fewer than 100 votes
# so we are only getting those titles that have the most engagement with fans on IMDB
full_df = full_df[(full_df.numvotes >= 100)]

In [21]:
# Finally, I will drop all movies from 2014 and earlier, so any insights we derive are based on only the most
# recent information
full_df.start_year.value_counts()

2017    3402
2016    3369
2015    3276
2014    3258
2013    3103
2012    2895
2018    2809
2011    2720
2010    2540
2019     606
Name: start_year, dtype: int64

In [22]:
full_df = full_df[(full_df.start_year >= 2015)]

In [23]:
full_df = full_df.reset_index()

In [24]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13462 entries, 0 to 13461
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            13462 non-null  int64  
 1   tconst           13462 non-null  object 
 2   primary_title    13462 non-null  object 
 3   original_title   13462 non-null  object 
 4   start_year       13462 non-null  int64  
 5   runtime_minutes  13462 non-null  float64
 6   genres           13451 non-null  object 
 7   averagerating    13462 non-null  float64
 8   numvotes         13462 non-null  float64
 9   actor            11877 non-null  object 
 10  actress          10775 non-null  object 
 11  composer         8352 non-null   object 
 12  director         12740 non-null  object 
 13  editor           4723 non-null   object 
 14  producer         10686 non-null  object 
 15  writer           7930 non-null   object 
dtypes: float64(3), int64(2), object(11)
memory usage: 1.6+ MB


## IMDbPY query:  Financials

### Performing the query

In [52]:
# SEE OTHER PROJECT FILE

### Joining and cleaning

In [25]:
## found this code online at:
## https://stackoverflow.com/questions/55424733/how-can-i-reformat-a-json-file-to-contain-an-array

json_filename = 'imdb_financials.json'

with open(json_filename) as file:
    array = {'foo': []}
    foo_list = array['foo']
    for line in file:
        obj = json.loads(line)
        foo_list.append(obj)

In [26]:
len(foo_list)

13462

In [27]:
full_df['financials'] = foo_list

In [28]:
full_df.head(10)

,index,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,actress,composer,director,editor,producer,writer,financials
0,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,"nm0001379, nm0000953","nm0462648, nm0001782",nm0006166,nm0000080,nm0613657,"nm0550881, nm1475059",NaN,0
1,4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0016013, nm0721280","nm0728971, nm1415193",nm0005948,"nm0749914, nm0765384",NaN,"nm0462571, nm1856792",nm1360635,0
2,7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263.0,"nm0365480, nm0003210","nm0367762, nm0186322",nm1930572,NaN,NaN,"nm0153581, nm0448515, nm0908708",NaN,0
3,38,tt0315642,Wazir,Wazir,2016,103.0,"Action,Crime,Drama",7.1,15378.0,"nm0000821, nm1027719, nm1303433",nm2390814,NaN,nm2349060,NaN,NaN,"nm3871075, nm0006765, nm0430785, nm5241801, nm...",0
4,47,tt0331314,Bunyan and Babe,Bunyan and Babe,2017,84.0,"Adventure,Animation,Comedy",5.0,302.0,"nm0000422, nm0289344, nm0001288, nm2259477",NaN,"nm1356349, nm0688953",nm8625898,NaN,nm1240210,"nm0630057, nm0908438",0
5,50,tt0337926,Chatô - The King of Brazil,Chatô: O Rei do Brasil,2015,102.0,Biography,6.0,558.0,"nm0723142, nm0079343","nm0069683, nm0754542",NaN,nm0285071,nm0479884,NaN,"nm0138676, nm0730422, nm1412474",0
6,51,tt0339736,The Evil Within,The Evil Within,2017,98.0,Horror,5.6,2420.0,"nm0001218, nm0462735, nm0077720",nm0000539,nm0339704,nm1274189,NaN,"nm0524451, nm0823123, nm2999181",NaN,"{'Budget': '$6,000,000 (estimated)'}"
7,55,tt0360556,Fahrenheit 451,Fahrenheit 451,2018,100.0,"Drama,Sci-Fi,Thriller",4.9,14469.0,"nm0430107, nm2210323, nm0788335",nm0441654,"nm1902248, nm0664020",nm1023919,NaN,nm0167708,"nm0001969, nm0618881",0
8,57,tt0365545,Nappily Ever After,Nappily Ever After,2018,98.0,"Comedy,Drama,Romance",6.4,6287.0,"nm1340638, nm0072713","nm0005125, nm0005551",NaN,nm2223783,NaN,"nm1545176, nm0082894","nm0111845, nm10021676, nm1946260",0
9,60,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338.0,"nm0695435, nm1339223","nm0397171, nm0339460",NaN,nm1119880,NaN,nm0189777,"nm0415425, nm0798646, nm2081046, nm0000341","{'Budget': '$150,000,000 (estimated)', 'Openin..."


In [29]:
# Getting the value_counts() for the new financials column returns an error, as the non-zero entries
# are all dictionaries

full_df['financials'].value_counts()

TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


0                                                                                                                                                 8961
{'Budget': '$1,000,000 (estimated)'}                                                                                                               104
{'Budget': '$500,000 (estimated)'}                                                                                                                  59
{'Budget': '$2,000,000 (estimated)'}                                                                                                                58
{'Budget': '$3,000,000 (estimated)'}                                                                                                                57
                                                                                                                                                  ... 
{'Budget': '$16,000,000 (estimated)', 'Opening Weekend United States': '$6,870,740, 17 Jun 201

In [30]:
# The results indicate 8961 items in full_df don't have associated financial data - so I will drop these 
# rows below

full_df = full_df[(full_df['financials'] != 0)]

In [31]:
len(full_df)

4501

In [33]:
full_df.head()

,index,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,actress,composer,director,editor,producer,writer,financials
6,51,tt0339736,The Evil Within,The Evil Within,2017,98.0,Horror,5.6,2420.0,"nm0001218, nm0462735, nm0077720",nm0000539,nm0339704,nm1274189,NaN,"nm0524451, nm0823123, nm2999181",NaN,"{'Budget': '$6,000,000 (estimated)'}"
9,60,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338.0,"nm0695435, nm1339223","nm0397171, nm0339460",NaN,nm1119880,NaN,nm0189777,"nm0415425, nm0798646, nm2081046, nm0000341","{'Budget': '$150,000,000 (estimated)', 'Openin..."
11,86,tt0420293,The Stanford Prison Experiment,The Stanford Prison Experiment,2015,122.0,"Biography,Drama,History",6.9,32591.0,"nm3009232, nm4446467, nm0001082",nm1880888,NaN,nm1547859,NaN,"nm0295288, nm2886189, nm0256283","nm0848003, nm1674354","{'Opening Weekend United States': '$37,514, 19..."
13,107,tt0437086,Alita: Battle Angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.5,88207.0,"nm0910607, nm0991810","nm4023073, nm0000124",nm0432725,nm0001675,NaN,nm0484457,"nm0000116, nm0436164, nm1738737","{'Budget': '$170,000,000 (estimated)', 'Openin..."
14,118,tt0443533,The History of Love,The History of Love,2016,134.0,"Drama,Romance,War",6.3,1024.0,"nm0001394, nm0001285","nm2605345, nm4563869",nm0023940,nm0586123,NaN,"nm2275877, nm2274042","nm0738925, nm1842569","{'Budget': 'EUR15,000,000 (estimated)'}"


In [34]:
full_df.reset_index(inplace=True)
full_df = full_df.drop(['level_0','index'], axis = 1)

In [35]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           4501 non-null   object 
 1   primary_title    4501 non-null   object 
 2   original_title   4501 non-null   object 
 3   start_year       4501 non-null   int64  
 4   runtime_minutes  4501 non-null   float64
 5   genres           4499 non-null   object 
 6   averagerating    4501 non-null   float64
 7   numvotes         4501 non-null   float64
 8   actor            4101 non-null   object 
 9   actress          3676 non-null   object 
 10  composer         2611 non-null   object 
 11  director         4244 non-null   object 
 12  editor           1294 non-null   object 
 13  producer         3691 non-null   object 
 14  writer           2967 non-null   object 
 15  financials       4501 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory usage: 562.8+ KB


In [36]:
full_df.financials

0                    {'Budget': '$6,000,000 (estimated)'}
1       {'Budget': '$150,000,000 (estimated)', 'Openin...
2       {'Opening Weekend United States': '$37,514, 19...
3       {'Budget': '$170,000,000 (estimated)', 'Openin...
4                 {'Budget': 'EUR15,000,000 (estimated)'}
                              ...                        
4496                 {'Budget': '$1,500,000 (estimated)'}
4497                 {'Budget': '$3,000,000 (estimated)'}
4498    {'Budget': 'MYR20,000,000 (estimated)', 'Cumul...
4499               {'Budget': 'INR4,000,000 (estimated)'}
4500              {'Budget': 'INR10,000,000 (estimated)'}
Name: financials, Length: 4501, dtype: object

In [37]:
# Found code online to extract multiple dict items into their own appropriately named columns:
# https://codereview.stackexchange.com/questions/93923/extracting-contents-of-dictionary-contained-in-pandas-dataframe-to-make-new-data

def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

In [38]:
full_df = unpack(full_df, 'financials', 0)

In [39]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 59 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   tconst                                4501 non-null   object 
 1   primary_title                         4501 non-null   object 
 2   original_title                        4501 non-null   object 
 3   start_year                            4501 non-null   int64  
 4   runtime_minutes                       4501 non-null   float64
 5   genres                                4499 non-null   object 
 6   averagerating                         4501 non-null   float64
 7   numvotes                              4501 non-null   float64
 8   actor                                 4101 non-null   object 
 9   actress                               3676 non-null   object 
 10  composer                              2611 non-null   object 
 11  director         

In [40]:
# Now I will drop columns 18 to the end, so I am left with just Budget, Opening Wkdn US, and Worldwide Gross 
full_df = full_df.iloc[:,0:18]
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tconst                         4501 non-null   object 
 1   primary_title                  4501 non-null   object 
 2   original_title                 4501 non-null   object 
 3   start_year                     4501 non-null   int64  
 4   runtime_minutes                4501 non-null   float64
 5   genres                         4499 non-null   object 
 6   averagerating                  4501 non-null   float64
 7   numvotes                       4501 non-null   float64
 8   actor                          4101 non-null   object 
 9   actress                        3676 non-null   object 
 10  composer                       2611 non-null   object 
 11  director                       4244 non-null   object 
 12  editor                         1294 non-null   o

In [41]:
full_df['Cumulative Worldwide Gross'].value_counts()

0                              3570
$90,000,000                       2
INR520,000,000                    2
$27,365,467, 23 Jul 2018          1
$20,000                           1
                               ... 
$1,389,814, 16 Jan 2017           1
$162,526,238                      1
$51,888, 08 Jan 2017              1
$16,810,562                       1
$1,332,539,889, 19 Apr 2018       1
Name: Cumulative Worldwide Gross, Length: 930, dtype: int64

In [42]:
# My primary measure of financial success will be via the 'Cumulative Worldwide Gross' column
# so I will drop all rows where there is no measure for this value

full_df = full_df[(full_df['Cumulative Worldwide Gross'] != 0)]

In [43]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931 entries, 1 to 4498
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tconst                         931 non-null    object 
 1   primary_title                  931 non-null    object 
 2   original_title                 931 non-null    object 
 3   start_year                     931 non-null    int64  
 4   runtime_minutes                931 non-null    float64
 5   genres                         931 non-null    object 
 6   averagerating                  931 non-null    float64
 7   numvotes                       931 non-null    float64
 8   actor                          882 non-null    object 
 9   actress                        802 non-null    object 
 10  composer                       478 non-null    object 
 11  director                       898 non-null    object 
 12  editor                         165 non-null    ob

In [44]:
# rename the columns
full_df.rename(columns = {'Budget':'budget', 
                          'Opening Weekend United States':'usa_opening_wknd', 
                          'Cumulative Worldwide Gross':'global_gross'}, inplace=True)

In [45]:
# It looked like some values in global_gross are not in USD, so I will try to find and drop those rows too
# There also appear to be some items with a date included, so I will need to isolate just the dollar values
full_df.global_gross.value_counts()

$90,000,000                      2
INR520,000,000                   2
$118,685,662                     1
$98,599,942                      1
INR1,500,000,000, 17 May 2016    1
                                ..
$10,815                          1
$37,357,216                      1
$1,389,814, 16 Jan 2017          1
$162,526,238                     1
$38,803,993                      1
Name: global_gross, Length: 929, dtype: int64

In [46]:
# splitting items with dates included
full_df['globalgross'] = full_df['global_gross'].map(lambda x: x.split(', '))

In [47]:
full_df.globalgross.value_counts

<bound method IndexOpsMixin.value_counts of 1       [$1,648,854,864, 04 Mar 2016]
3                      [$404,852,543]
5                      [$365,971,656]
6                      [$821,763,408]
7             [$494,123, 20 Aug 2017]
                    ...              
4475    [PHP600,000,000, 30 Jan 2019]
4485                    [$94,951,615]
4488    [PHP370,000,000, 13 Mar 2019]
4491                   [$255,863,112]
4498     [MYR25,280,000, 27 May 2019]
Name: globalgross, Length: 931, dtype: object>

In [48]:
# Any items that the split was applied to were turned into 2-item lists
# I need to keep just the monetary value component of those items and drop the date

globgross = []

for i in full_df.globalgross:
        globgross.append(str(i[0]))

In [49]:
len(full_df['global_gross']) == len(globgross)

True

In [50]:
# now I will apply my cleaned column values within the globgross list to my full_df dataframe

full_df['global_gross'] = globgross

In [51]:
full_df = full_df.drop('globalgross', axis=1)

In [52]:
full_df['global_gross'].value_counts()

INR200,000,000    4
INR520,000,000    3
$3,500,000        3
$1,000,000        2
INR120,000,000    2
                 ..
$3,164,333        1
$371,985,018      1
$494,123          1
MYR25,280,000     1
$38,803,993       1
Name: global_gross, Length: 919, dtype: int64

In [53]:
# Using regex, I will filter for all rows that start with a non-alphanumeric character
full_df = full_df[(full_df.global_gross.str.match('(\W)')==True)]

In [54]:
full_df.reset_index(inplace=True)

In [55]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             833 non-null    int64  
 1   tconst            833 non-null    object 
 2   primary_title     833 non-null    object 
 3   original_title    833 non-null    object 
 4   start_year        833 non-null    int64  
 5   runtime_minutes   833 non-null    float64
 6   genres            833 non-null    object 
 7   averagerating     833 non-null    float64
 8   numvotes          833 non-null    float64
 9   actor             785 non-null    object 
 10  actress           716 non-null    object 
 11  composer          396 non-null    object 
 12  director          806 non-null    object 
 13  editor            122 non-null    object 
 14  producer          694 non-null    object 
 15  writer            675 non-null    object 
 16  budget            833 non-null    object 
 1

In [56]:
full_df.global_gross.value_counts()

$3,500,000      3
$20,000         2
$90,000,000     2
$1,000,000      2
$557,969        1
               ..
$386,600,138    1
$6,553,762      1
$3,164,333      1
$371,985,018    1
$38,803,993     1
Name: global_gross, Length: 828, dtype: int64

In [57]:
full_df.to_csv('full_df_3.5_883rows')

## IMDbPY query: Add'l metadata

### Performing the query

In [90]:
# Per section 3.1, I will pull the following additional metadata and rating features using IMDbPY

# 1. MOVIE RATING: IMDbPY 'main', 'certificates'
# 2. PRODUCTION STUDIO: IMDbPY 'main', 'production companies'
# 3. VOTING DEMOGRAPHICS: 'vote details', 'demographics'

In [91]:
# SEE OTHER PROJECT FILE

### Joining & cleaning: Ratings

In [58]:
with open(r"imdb_ratings.json", "r") as read_file:
    ratings_json = json.load(read_file)

In [59]:
# Quick exploration of the file imdb_ratings.json
ratings_json[0]

['Argentina:13::(with warning)',
 'Australia:M',
 'Austria:12',
 'Brazil:12',
 'Canada:PG::(British Columbia)',
 'Canada:PG::(tv rating)',
 'Chile:TE+7',
 'Colombia:T',
 'Denmark:11',
 'Finland:K-12',
 'France:Tous publics::(with warning)',
 'Germany:12',
 'Greece:K-13',
 'Hong Kong:IIA',
 'Hungary:12',
 'Hungary:16',
 'Iceland:12',
 'India:UA',
 'Ireland:12A',
 'Ireland:12',
 'Italy:T',
 'Japan:G',
 'Lithuania:N-13',
 'Malaysia:P13',
 'Maldives:15+',
 'Mexico:B',
 'Netherlands:12',
 'New Zealand:M',
 'Norway:12',
 'Norway:12::(cinema rating)',
 'Philippines:PG-13',
 'Poland:12',
 'Portugal:M/12',
 'Russia:12+',
 'Russia:16+::(DVD rating)',
 'Singapore:PG13',
 'South Africa:10-12',
 'South Korea:12',
 'Spain:12',
 'Spain:16::(Movistar+)',
 'Sweden:11',
 'Switzerland:12',
 'Thailand:G',
 'Thailand:13::(DVD rating)',
 'United Kingdom:12A',
 'United Kingdom:12',
 'United States:TV-14::(LV)',
 'United States:PG-13',
 'Vietnam:P::(2015, self-applied)']

In [60]:
ratings_json[0][0]

'Argentina:13::(with warning)'

In [61]:
# Code that iterates through imdb_rating, finds and extracts the USA rating for each film
# Some films have two ratings listed for the USA, so code needs to associate both of these with the same row #

countries = []
ratings = []
rownums = []

for x, i in enumerate(ratings_json):
    if i == 0:
        continue
    else:
        for item in i:
            country = str(item).split(':')[0]
            if country != "United States":
                continue
            else:
                countries.append(item.split(":")[0])
                ratings.append(item.split(":")[1])
            rownums.append(x)

In [62]:
# Zipping row numbers and ratings together, so each rating is associated with the correct row in full_df
ratingdata = list(zip(rownums, ratings))

In [63]:
ratingdata[:10]

[(0, 'TV-14'),
 (0, 'PG-13'),
 (1, 'PG-13'),
 (2, 'PG-13'),
 (3, 'PG-13'),
 (4, 'R'),
 (5, 'PG-13'),
 (6, 'PG-13'),
 (7, 'R'),
 (8, 'PG-13')]

In [64]:
# Remove duplicate ratings for any film that had both TV and movie ratings listed

movieratings = []
for item in ratingdata:
    if item[1].split('-')[0] == 'TV':
        continue
    else:
        movieratings.append(item)

In [65]:
# Split row number and rating into their own separate lists, in prep for conversion to dataframe
row = []
rating = []

for i in movieratings:
    row.append(i[0])
    rating.append(i[1])

seriesrow = pd.Series(row)
seriesratings = pd.Series(rating)

In [66]:
#convert to dictionary, for two column dataframe
ratingdict = {'rownum':row, 'movrating':rating}

In [67]:
df_movieratings = pd.DataFrame(ratingdict)

In [68]:
df_movieratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rownum     731 non-null    int64 
 1   movrating  731 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.5+ KB


In [71]:
df_movieratings.tail()

,rownum,movrating
726,823,R
727,825,G
728,826,Not Rated
729,828,PG
730,829,Unrated


In [89]:
df_movieratings['rownum'].duplicated().sum()

11

In [96]:
len(df_movieratings[~df_movieratings['rownum'].duplicated()])

720

In [97]:
df_movieratings = df_movieratings[~df_movieratings['rownum'].duplicated()]

In [98]:
df_movieratings['rownum'].value_counts()

829    1
251    1
249    1
248    1
247    1
      ..
508    1
507    1
506    1
505    1
0      1
Name: rownum, Length: 720, dtype: int64

In [72]:
full_df.tail()

,index,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,actress,composer,director,editor,producer,writer,budget,usa_opening_wknd,global_gross
828,4441,tt8969332,The Biggest Little Farm,The Biggest Little Farm,2018,91.0,Documentary,8.0,421.0,nm10726590,NaN,nm0063618,NaN,NaN,nm3537480,nm0598531,0,"$110,492, 12 May 2019","$5,293,304"
829,4442,tt8991268,Honeyland,Honeyland,2019,87.0,Documentary,8.5,158.0,NaN,NaN,nm3334929,"nm9393813, nm10118100",NaN,nm1389493,NaN,0,0,"$1,140,879"
830,4456,tt9081562,More Than Blue,Bi bei shang geng bei shang de gu shi,2018,105.0,Romance,5.6,566.0,"nm6630432, nm3214105","nm2442121, nm5397122","nm3871161, nm3928207",nm4341096,NaN,nm7599343,nm7250425,0,0,"$153,000,000"
831,4485,tt9408490,Kill Mobile,Shoujikuang xiang,2018,90.0,Drama,5.7,198.0,"nm1291827, nm4596259","nm4454684, nm2356251",NaN,nm6785661,NaN,NaN,"nm2601560, nm3798121, nm0182499, nm0002645, nm...",0,0,"$94,951,615"
832,4491,tt9597190,Pegasus,Fei chi ren sheng,2019,98.0,"Comedy,Sport",6.4,817.0,"nm7613067, nm8361677, nm7613069, nm8288748",NaN,nm0150989,nm3954274,NaN,"nm10464217, nm7815808, nm7815809, nm10464218",NaN,0,0,"$255,863,112"


In [73]:
# Replacing index column with a new index column from 0 to 832, in order to join in the ratings data
full_df = full_df.drop('index', axis=1)
full_df = full_df.reset_index()

In [99]:
full_df.tail()

,index,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,actress,composer,director,editor,producer,writer,budget,usa_opening_wknd,global_gross
828,828,tt8969332,The Biggest Little Farm,The Biggest Little Farm,2018,91.0,Documentary,8.0,421.0,nm10726590,NaN,nm0063618,NaN,NaN,nm3537480,nm0598531,0,"$110,492, 12 May 2019","$5,293,304"
829,829,tt8991268,Honeyland,Honeyland,2019,87.0,Documentary,8.5,158.0,NaN,NaN,nm3334929,"nm9393813, nm10118100",NaN,nm1389493,NaN,0,0,"$1,140,879"
830,830,tt9081562,More Than Blue,Bi bei shang geng bei shang de gu shi,2018,105.0,Romance,5.6,566.0,"nm6630432, nm3214105","nm2442121, nm5397122","nm3871161, nm3928207",nm4341096,NaN,nm7599343,nm7250425,0,0,"$153,000,000"
831,831,tt9408490,Kill Mobile,Shoujikuang xiang,2018,90.0,Drama,5.7,198.0,"nm1291827, nm4596259","nm4454684, nm2356251",NaN,nm6785661,NaN,NaN,"nm2601560, nm3798121, nm0182499, nm0002645, nm...",0,0,"$94,951,615"
832,832,tt9597190,Pegasus,Fei chi ren sheng,2019,98.0,"Comedy,Sport",6.4,817.0,"nm7613067, nm8361677, nm7613069, nm8288748",NaN,nm0150989,nm3954274,NaN,"nm10464217, nm7815808, nm7815809, nm10464218",NaN,0,0,"$255,863,112"


In [103]:
full_df = full_df.merge(df_movieratings, left_on='index', right_on='rownum', how='left')

In [104]:
full_df.tail()

,index,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,actor,...,composer,director,editor,producer,writer,budget,usa_opening_wknd,global_gross,rownum,movrating
828,828,tt8969332,The Biggest Little Farm,The Biggest Little Farm,2018,91.0,Documentary,8.0,421.0,nm10726590,...,nm0063618,NaN,NaN,nm3537480,nm0598531,0,"$110,492, 12 May 2019","$5,293,304",828.0,PG
829,829,tt8991268,Honeyland,Honeyland,2019,87.0,Documentary,8.5,158.0,NaN,...,nm3334929,"nm9393813, nm10118100",NaN,nm1389493,NaN,0,0,"$1,140,879",829.0,Unrated
830,830,tt9081562,More Than Blue,Bi bei shang geng bei shang de gu shi,2018,105.0,Romance,5.6,566.0,"nm6630432, nm3214105",...,"nm3871161, nm3928207",nm4341096,NaN,nm7599343,nm7250425,0,0,"$153,000,000",NaN,NaN
831,831,tt9408490,Kill Mobile,Shoujikuang xiang,2018,90.0,Drama,5.7,198.0,"nm1291827, nm4596259",...,NaN,nm6785661,NaN,NaN,"nm2601560, nm3798121, nm0182499, nm0002645, nm...",0,0,"$94,951,615",NaN,NaN
832,832,tt9597190,Pegasus,Fei chi ren sheng,2019,98.0,"Comedy,Sport",6.4,817.0,"nm7613067, nm8361677, nm7613069, nm8288748",...,nm0150989,nm3954274,NaN,"nm10464217, nm7815808, nm7815809, nm10464218",NaN,0,0,"$255,863,112",NaN,NaN


In [105]:
full_df = full_df.drop('rownum', axis=1)

In [106]:
full_df.to_csv('full_df_ratings.csv')

### Joining & cleaning: Voter demographics

In [107]:
with open(r"imdb_voterdemos.json", "r") as read_file:
    voters_json = json.load(read_file)

In [109]:
len(voters_json)

833